In [ ]:
import torch
import torch.nn as nn
from torchvision.models.detection import fasterrcnn_mobilenet_v3_large_fpn, FasterRCNN_MobileNet_V3_Large_FPN_Weights
from torchvision.datasets import CocoDetection
from torch.utils.data import DataLoader
from pycocotools.cocoeval import COCOeval
import torchvision
import torch.nn.functional as F

import copy
import json
import os
import time
from datetime import datetime

import matplotlib.pyplot as plt

In [ ]:
from bloc_diag_model.BlocDiagBoxHead import BlocDiagBoxHead
from bloc_diag_model.StructuralBlocDiagBoxHead import StructuralBlocDiagBoxHead
from utils.train_utils import train_one_epoch

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Load pretrained weights and model
weights = FasterRCNN_MobileNet_V3_Large_FPN_Weights.DEFAULT
model = fasterrcnn_mobilenet_v3_large_fpn(weights=weights)

In [ ]:
in_features = model.roi_heads.box_head.fc6.in_features
num_classes = 91


original_box_head = model.roi_heads.box_head

custom_box_head = BlocDiagBoxHead([12544, 1024, 1024], [[64]*16, [1024]], [[784]*16, [1024]], True)

custom_box_head.fc6.load_state_dict(original_box_head.fc6.state_dict())
custom_box_head.fc7.load_state_dict(original_box_head.fc7.state_dict())

model.roi_heads.box_head = custom_box_head.to(device)

### Load trained model with block-diagonal BoxHead

In [ ]:
n_total_epoch = 3
lambda_offdiag = 1e-4
model_timestamp_start = '2025-04-28_14-16-04'

model_save_path = os.path.join('./saved_models',
                               f"model_BlocDiagBoxHead__lambda_offdiag={lambda_offdiag}__n_total_epoch={n_total_epoch}__{model_timestamp_start}.pt")

In [ ]:
model.load_state_dict(torch.load(model_save_path))
model.to(device)

In [ ]:
fc6_weight = model.roi_heads.box_head.fc6.weight.data.clone()
fc6_bias = model.roi_heads.box_head.fc6.bias.data.clone()
fc7_weight = model.roi_heads.box_head.fc7.weight.data.clone()
fc7_bias = model.roi_heads.box_head.fc7.bias.data.clone()

In [ ]:
fc6_weight_cpu = fc6_weight.detach().cpu().numpy()

# Plot the weight matrix
plt.figure(figsize=(14, 6))
plt.imshow(fc6_weight_cpu, aspect='auto', interpolation='nearest', cmap='viridis', vmin=-0.01, vmax=0.01)
plt.colorbar(label='Weight value')
plt.title('fc6 Weight Matrix')
plt.xlabel('Input Features')
plt.ylabel('Output Neurons')
plt.show()

### Load weights into structural block-diagonal model

In [ ]:
custom_structural_box_head = StructuralBlocDiagBoxHead([12544, 1024, 1024], [[64]*16, [1024]], [[784]*16, [1024]], True)
# Load the weights into the new head
custom_structural_box_head.fc6.set_blocks_from_full_matrix(fc6_weight, fc6_bias)
custom_structural_box_head.fc7.set_blocks_from_full_matrix(fc7_weight, fc7_bias)

In [ ]:
model.roi_heads.box_head = custom_structural_box_head.to(device)
model.to(device)

In [ ]:
# Load the official transform used during pretraining
transform = weights.transforms()

In [ ]:
data_dir = "data/coco"
train_img_folder = os.path.join(data_dir, "train2017")
train_ann_file = os.path.join(data_dir, "annotations/instances_train2017.json")

In [ ]:
N = 20000
subset_indices = list(range(N))
dataset_raw = torch.utils.data.Subset(CocoDetection(train_img_folder, train_ann_file), subset_indices)
dataset = []
for i in range(N):
    img, target = dataset_raw[i]
    img_id = dataset_raw.dataset.ids[dataset_raw.indices[i]]
    transformed_img = transform(img)
    for t in target:
        t["image_id"] = img_id
    dataset.append((transformed_img, target))

In [ ]:
def coco_collate_fn(batch):
    return tuple(zip(*batch))

In [ ]:
data_loader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=coco_collate_fn)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)

In [ ]:
n_total_epoch = 2
timestamp_start = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')


for epoch_num in range(n_total_epoch):
    start_time = time.time()
    
    train_one_epoch(model, data_loader, optimizer, epoch_num=epoch_num+1, n_total_epoch=n_total_epoch,
                    device=device,
                    metrics_path=os.path.join('./saved_models', f"fine_tune_metrics__{timestamp_start}.csv"),
                    lambda_offdiag=None)

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Epoch {epoch_num+1}/{n_total_epoch} completed in {elapsed_time:.2f} seconds.")


model_save_path = os.path.join('./saved_models',
                               f"model_StructuralBlocDiagBoxHead__n_total_epoch={n_total_epoch}__{timestamp_start}.pt")
torch.save(model.state_dict(), model_save_path)

#### Fine-tuned model evaluation

In [ ]:
custom_structural_box_head = StructuralBlocDiagBoxHead([12544, 1024, 1024], [[64]*16, [1024]], [[784]*16, [1024]], True)

In [ ]:
model.roi_heads.box_head = custom_structural_box_head.to(device)
model.to(device)

In [ ]:
n_total_epoch = 2
model_timestamp_start = '2025-04-29_23-29-17'

model_save_path = os.path.join('./saved_models',
                               f"model_StructuralBlocDiagBoxHead__n_total_epoch={n_total_epoch}__{model_timestamp_start}.pt")

In [ ]:
model.load_state_dict(torch.load(model_save_path))
model.to(device)

In [ ]:
from utils.eval_utils import evaluate_model

In [ ]:
# Load the transform used during pretraining
transform = weights.transforms()

In [ ]:
# Define paths to COCO val2017
data_dir = "data/coco"
val_img_folder = os.path.join(data_dir, "val2017")
val_ann_file = os.path.join(data_dir, "annotations/instances_val2017.json")

In [ ]:
def coco_collate_fn(batch):
    return tuple(zip(*batch))

In [ ]:
# Load dataset and apply transform manually to images only
dataset_raw = CocoDetection(val_img_folder, val_ann_file)

In [ ]:
dataset = []
for i in range(len(dataset_raw)):
    img, target = dataset_raw[i]
    img_id = dataset_raw.ids[i]
    transformed_img = transform(img)
    for t in target:
        t["image_id"] = img_id
    dataset.append((transformed_img, target))

val_data_loader = DataLoader(dataset, batch_size=8, shuffle=False, collate_fn=coco_collate_fn)

In [ ]:
# Evaluate using pycocotools
coco_gt = dataset_raw.coco

In [ ]:
eval_file_path = os.path.join('./model_eval_results', f"coco_val_results__fine_tuned_model__{model_timestamp_start}.csv")

In [ ]:
evaluate_model(model, val_data_loader, device=device,
               output_path=eval_file_path)

In [ ]:
coco_dt = coco_gt.loadRes(eval_file_path)
eval = COCOeval(coco_gt, coco_dt, iouType='bbox')
eval.evaluate()
eval.accumulate()
eval.summarize()

### View the weight of the structural block-diagonal layer

In [ ]:
fc6_weight = model.roi_heads.box_head.fc6.get_full_matrix()
#fc6_bias = model.roi_heads.box_head.fc6.bias.clone()
fc7_weight = model.roi_heads.box_head.fc7.get_full_matrix()
#fc7_bias = model.roi_heads.box_head.fc7.bias.clone()

In [ ]:
plt.figure(figsize=(14, 6))
plt.imshow(fc6_weight, aspect='auto', interpolation='nearest', cmap='viridis', vmin=-0.01, vmax=0.01)
plt.colorbar(label="Weight Value")
plt.title("Weight Matrix")
plt.xlabel("Input Features")
plt.ylabel("Output Features")
plt.show()